<a href="https://colab.research.google.com/github/imai-ry/colabo/blob/main/AISprediction20210601.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AIS進路予測
hamakanaya-ais.log-161

In [6]:
# AISデコードライブラリのインストール
!python3 -m pip install libais

# 地図表示ライブラリ
!python3 -m pip install folium

ERROR: Could not find a version that satisfies the requirement basemap (from versions: none)
ERROR: No matching distribution found for basemap


In [2]:
# AISデータの保存場所指定

HOME = '/content/drive/MyDrive/colab-data/'
path = HOME + "hamakanaya-ais.log-1617034621"
print(path)

/content/drive/MyDrive/colab-data/hamakanaya-ais.log-1617034621


In [3]:
# AISデータの読み込み

import pandas as pd

rtime1 = []
rtime2 = []
raw_msgs = []

with open(path) as f:
  sl = f.readlines()
  for s in sl:
    s = s.split('\t')
    rtime1.append(s[0])
    rtime2.append(s[1])
    raw_msgs.append(s[2].strip('\n'))

d = {'rtime1': rtime1, "rtime2": rtime2, "raw_msg": raw_msgs}
df = pd.DataFrame(data=d)
df

,rtime1,rtime2,raw_msg
0,2021:03:29:15:17:02:859946,2021:03:30:00:17:02:859946,"!BSVDM,2,1,4,B,56KJU`h2;0WD`OG??>0DUL60l59F0pv..."
1,2021:03:29:15:17:02:886122,2021:03:30:00:17:02:886122,"!BSVDM,2,2,4,B,08jT8556H88888888888880,2*72"
2,2021:03:29:15:17:03:068219,2021:03:30:00:17:03:068219,"!BSVDM,1,1,,A,16KPivP01P9wTd`D6WvH;Vb620S1,0*61"
3,2021:03:29:15:17:03:518695,2021:03:30:00:17:03:518695,"!BSVDM,1,1,,A,16K2:?00019wTM>D9lHpC6<80821,0*7F"
4,2021:03:29:15:17:03:704913,2021:03:30:00:17:03:704913,"!BSVDM,1,1,,B,16K2Wb001M9wJwJD66R8pG<80@28,0*55"
...,...,...,...
12578,2021:03:29:16:17:00:379724,2021:03:30:01:17:00:379724,"!BSVDM,1,1,,B,16K8Dr@0019wU;rD9lA8vo2200Rr,0*0C"
12579,2021:03:29:16:17:00:505373,2021:03:30:01:17:00:505373,"!BSVDM,1,1,,A,36K2=G@Oin9wf2DD5E=P`0Ml00u1,0*03"
12580,2021:03:29:16:17:00:531470,2021:03:30:01:17:00:531470,"!BSVDM,1,1,,A,16K2<Ph01T9wbAdD4kHWJ6220<2r,0*2D"
12581,2021:03:29:16:17:00:640902,2021:03:30:01:17:00:640902,"!BSVDM,1,1,,A,16K2twh01t9wlftD:GEG05cn0D7I,0*56"


In [4]:
# AISのデコード
%cd $HOME
from ais_decoder.ais_decoder import AisDecoder
import numpy as np

decoder = AisDecoder(["!BSVDM"])
nav_data = {
    "id": [],
    "mmsi": [],
    "sog": [],
    "x": [],
    "y": [],
    "cog": [],
    "true_heading": [],
}

for raw_msg in df["raw_msg"]:
    out = decoder.decode(raw_msg)
    decoded = out["decoded"]
    if not decoded:
        continue
    for key in nav_data.keys():
        val = decoded[key] if (key in decoded) else np.nan
        nav_data[key].append(val) 

ais_df = pd.DataFrame(data=nav_data)
ais_df


/content/drive/MyDrive/colab-data


,id,mmsi,sog,x,y,cog,true_heading
0,5,431400355,NaN,NaN,NaN,NaN,NaN
1,1,431501818,9.6,139.716940,35.133428,209.399994,213.0
2,1,431000124,0.1,139.716118,35.220645,212.399994,198.0
3,1,431007656,9.3,139.683808,35.119160,227.300003,230.0
4,1,431000742,0.1,139.713855,35.222645,351.600006,352.0
...,...,...,...,...,...,...,...
10932,1,431101161,0.1,139.718608,35.220593,229.899994,225.0
10933,3,431000925,11.8,139.748817,35.098117,16.000000,14.0
10934,1,431000707,10.0,139.735983,35.083683,189.600006,193.0
10935,1,431013119,12.4,139.771677,35.235555,179.199997,181.0


In [95]:
# 特定のMMSI & id 1~3のみの列を抽出
target = ais_df[ais_df["mmsi"]==431501818]

# nan除去
target_n = target.dropna(how="any")

# xyのみ抽出
xy = target_n.loc[:, ["x","y"]]

# 緯度経度を入れ替えて、pythonリストに変換
latlngs = xy.values[:,[1,0]].tolist()
latlngs[:5]

[[35.133428333333335, 139.71694],
 [35.133048333333335, 139.71667],
 [35.13270166666667, 139.71641666666667],
 [35.13234333333333, 139.71618],
 [35.13191666666667, 139.71588166666666]]

In [94]:
# 航跡の表示
import folium

m = folium.Map(location=[35.3, 140], zoom_start=10)

polyline = folium.PolyLine(latlngs, color="red")
polyline.add_to(m)
m